In [1]:
import pandas as pd
import numpy as np

In [2]:
from keras.models import Sequential
from keras.layers import Dense, Activation,Layer,Lambda

In [5]:
 forestfires = pd.read_csv("forestfires.csv")

In [6]:
forestfires.drop(["month","day"],axis=1,inplace = True)

In [7]:
forestfires["size_category"].value_counts()

small    378
large    139
Name: size_category, dtype: int64

In [8]:
forestfires.isnull().sum()

FFMC             0
DMC              0
DC               0
ISI              0
temp             0
RH               0
wind             0
rain             0
area             0
dayfri           0
daymon           0
daysat           0
daysun           0
daythu           0
daytue           0
daywed           0
monthapr         0
monthaug         0
monthdec         0
monthfeb         0
monthjan         0
monthjul         0
monthjun         0
monthmar         0
monthmay         0
monthnov         0
monthoct         0
monthsep         0
size_category    0
dtype: int64

In [9]:
forestfires.describe()

,FFMC,DMC,DC,ISI,temp,RH,wind,rain,area,dayfri,daymon,daysat,daysun,daythu,daytue,daywed,monthapr,monthaug,monthdec,monthfeb,monthjan,monthjul,monthjun,monthmar,monthmay,monthnov,monthoct,monthsep
count,517.000000,517.000000,517.000000,517.000000,517.000000,517.000000,517.000000,517.000000,517.000000,517.000000,517.000000,517.000000,517.000000,517.000000,517.000000,517.000000,517.000000,517.000000,517.000000,517.000000,517.000000,517.000000,517.000000,517.000000,517.000000,517.000000,517.000000,517.000000
mean,90.644681,110.872340,547.940039,9.021663,18.889168,44.288201,4.017602,0.021663,12.847292,0.164410,0.143133,0.162476,0.183752,0.117988,0.123791,0.104449,0.017408,0.355899,0.017408,0.038685,0.003868,0.061896,0.032882,0.104449,0.003868,0.001934,0.029014,0.332689
std,5.520111,64.046482,248.066192,4.559477,5.806625,16.317469,1.791653,0.295959,63.655818,0.371006,0.350548,0.369244,0.387657,0.322907,0.329662,0.306138,0.130913,0.479249,0.130913,0.193029,0.062137,0.241199,0.178500,0.306138,0.062137,0.043980,0.168007,0.471632
min,18.700000,1.100000,7.900000,0.000000,2.200000,15.000000,0.400000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,90.200000,68.600000,437.700000,6.500000,15.500000,33.000000,2.700000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,91.600000,108.300000,664.200000,8.400000,19.300000,42.000000,4.000000,0.000000,0.520000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,92.900000,142.400000,713.900000,10.800000,22.800000,53.000000,4.900000,0.000000,6.570000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
max,96.200000,291.300000,860.600000,56.100000,33.300000,100.000000,9.400000,6.400000,1090.840000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [10]:
forestfires.loc[forestfires["size_category"]=='small','size_category']=0

In [11]:
forestfires.loc[forestfires["size_category"]=='large','size_category']=1

In [12]:
forestfires["size_category"].value_counts()

0    378
1    139
Name: size_category, dtype: int64

In [13]:
def norm_func(i):
     x = (i-i.min())	/	(i.max()	-	i.min())
     return (x)

In [14]:
predictors = forestfires.iloc[:,0:28]

In [15]:
target = forestfires.iloc[:,28]

In [16]:
predictors1 = norm_func(predictors)

In [17]:
from sklearn.model_selection import train_test_split

In [18]:
x_train,x_test,y_train,y_test= train_test_split(predictors1,target, test_size=0.3,stratify = target)

In [19]:
def prep_model(hidden_dim):
    model = Sequential()
    for i in range(1,len(hidden_dim)-1):
        if (i==1):
            model.add(Dense(hidden_dim[i],input_dim=hidden_dim[0],activation="relu"))
        else:
            model.add(Dense(hidden_dim[i],activation="relu"))
    model.add(Dense(hidden_dim[-1],kernel_initializer="normal",activation="sigmoid"))
    model.compile(loss="binary_crossentropy",optimizer = "rmsprop",metrics = ["accuracy"])
    return model

In [24]:
  y_train = pd.DataFrame(y_train)

In [28]:
y_train

,size_category
122,0
411,0
315,0
244,0
418,0
...,...
59,0
457,1
335,0
490,0


In [29]:
x_train

,FFMC,DMC,DC,ISI,temp,RH,wind,rain,area,dayfri,daymon,daysat,daysun,daythu,daytue,daywed,monthapr,monthaug,monthdec,monthfeb,monthjan,monthjul,monthjun,monthmar,monthmay,monthnov,monthoct,monthsep
122,0.950968,0.423846,0.789023,0.151515,0.652733,0.317647,0.555556,0.0,0.000000,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
411,0.850323,0.007236,0.041867,0.058824,0.192926,0.447059,1.000000,0.0,0.004235,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
315,0.935484,0.460372,0.949455,0.128342,0.524116,0.176471,0.255556,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
244,0.943226,0.599586,0.812478,0.245989,0.649518,0.458824,0.800000,0.0,0.002631,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
418,0.945806,0.696416,0.770025,0.144385,0.729904,0.317647,0.555556,0.0,0.002237,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59,0.876129,0.041695,0.041163,0.094474,0.324759,0.423529,0.055556,0.0,0.000000,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
457,0.941935,0.655755,0.736484,0.139037,0.569132,0.411765,0.400000,0.0,0.075859,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
335,0.940645,0.369745,0.886713,0.110517,0.244373,0.835294,0.155556,0.0,0.000000,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
490,0.985806,0.483115,0.701184,0.315508,0.852090,0.211765,0.400000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [26]:
first_model = prep_model([28,50,40,20,1])

In [27]:
first_model.fit(np.array(x_train),np.array(y_train),epochs=500)

ValueError: ignored

In [31]:
x = np.asarray(x).astype('float32')

NameError: ignored

In [32]:
pred_train = first_model.predict(np.array(x_train))

In [33]:
pred_train = pd.Series([i[0] for i in pred_train])

In [34]:
size = ["small","large"]

In [35]:
pred_train_class = pd.Series(["small"]*361)

In [36]:
pred_train_class[[i>0.5 for i in pred_train]]= "large"

In [37]:
train = pd.concat([x_train,y_train],axis=1)

In [38]:
train["size_category"].value_counts()

0    264
1     97
Name: size_category, dtype: int64

In [39]:
from sklearn.metrics import confusion_matrix

In [40]:
train["original_class"] = "small"

In [41]:
train.loc[train["size_category"]==1,"original_class"] = "large"

In [42]:
train.original_class.value_counts()

small    264
large     97
Name: original_class, dtype: int64

In [43]:
confusion_matrix(pred_train_class,train["original_class"])

array([[  0,   0],
       [ 97, 264]])

In [44]:
np.mean(pred_train_class==pd.Series(train["original_class"]).reset_index(drop=True))

0.7313019390581718

In [45]:
pd.crosstab(pred_train_class,pd.Series(train["original_class"]).reset_index(drop=True))

original_class,large,small
row_0,,
small,97,264


In [46]:
pred_test = first_model.predict(np.array(x_test))

In [47]:
pred_test = pd.Series([i[0] for i in pred_test])

In [48]:
pred_test_class = pd.Series(["small"]*156)

In [49]:
pred_test_class[[i>0.5 for i in pred_test]] = "large"

In [50]:
test =pd.concat([x_test,y_test],axis=1)

In [51]:
test["original_class"]="small"

In [52]:
test.loc[test["size_category"]==1,"original_class"] = "large"

In [53]:
test["original_class"].value_counts()

small    114
large     42
Name: original_class, dtype: int64

In [54]:
np.mean(pred_test_class==pd.Series(test["original_class"]).reset_index(drop=True))

0.7307692307692307

In [55]:
confusion_matrix(pred_test_class,test["original_class"])

array([[  0,   0],
       [ 42, 114]])

In [56]:
pd.crosstab(pred_test_class,pd.Series(test["original_class"]).reset_index(drop=True))

original_class,large,small
row_0,,
small,42,114
